In [5]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
import yfinance as yf

In [6]:
windows_size = 70       # Onceki 6 degeri kullan
output_size = 2         # sonraki 4 degeri tahmin etmeye calis
batch_size = 8          # her bir batinda kac satirda islem yapayim
epochs = 700            # islemi tekrar sayisi

In [7]:
def get_data(y):

    train_size = int(len(y)*0.7)                # Verinin %70 ini egitim icin kalaninin da test icin ayiracagiz, 180

    ############## TRAIN DATA ####################
    train_x = []
    train_y = []
    for i in range(0, train_size):
        train_x.append(y[i:i + windows_size])
        train_y.append((y[i + windows_size:i + windows_size + output_size]))

    train_x = np.array(train_x)
    train_y = np.array(train_y)

    ########### TEST DATA ########################
    test_x = []
    test_y = []
    last = len(y) - output_size - windows_size
    for i in range(train_size, last):
        test_x.append(y[i:i + windows_size])
        test_y.append(y[i + windows_size:i + windows_size + output_size])

    test_x = np.array(test_x)
    test_y = np.array(test_y)

    ######## Tahmin edilecek data #######################
    data_x = [y[-windows_size:len(y)]]
    data_x = np.array(data_x)

    return train_x, train_y, test_x, test_y, data_x

In [8]:
df_btc = yf.download('BTC-USD', period='3mo', interval='1h')

[*********************100%***********************]  1 of 1 completed


In [9]:
t = np.array(df_btc.index.values)
y = np.array(df_btc.Close.values)

In [10]:
min = y.min()
max = y.max()

In [11]:
y = np.interp(y, (min, max), (-1, +1))

x_train, y_train, x_test, y_test, data_x = get_data(y)

In [12]:
model = Sequential()
model.add(Dense(32, input_dim=windows_size, activation='relu'))
model.add(Dense(64))
model.add(Dense(output_size))

2022-09-22 21:29:10.598903: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-22 21:29:10.624400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-22 21:29:10.624443: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-22 21:29:10.624989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [ ]:
model.compile(loss='mse', optimizer='rmsprop', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=epochs, verbose=0, batch_size=batch_size)

In [14]:
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print("%2s: %.2f%%" % (model.metrics_names[1], score[1]*100))
model.summary()

74/74 [==============================] - 0s 3ms/step - loss: 0.0055 - accuracy: 0.4975
accuracy: 49.75%
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                2272      
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 4,514
Trainable params: 4,514
Non-trainable params: 0
_________________________________________________________________


In [ ]:
data_y = model.predict(data_x)
result = np.interp(data_y, (-1, +1), (min, max))
print("Gelecekteki Degerler (output_size) :", result)